In [232]:
import pandas as pd
import json
import re
from functools import reduce

## Data

In [2]:
pd.set_option('display.max_colwidth', None)

In [3]:
resume_frame = pd.read_csv('resume_to_hackaton.csv', delimiter=';')
vac_frame = pd.read_csv('vacancies_to_hackaton.csv', delimiter=';')

In [162]:
display(resume_frame)

,uuid,position,organization,description,start,end
0,e08588cc-4ad1-45be-ae95-fa3fd31bc32e,заведующая кафе-кладовщик,ООО Мираж,Составление ттк\r Проведение инвентаризаций\r Разработка и составление меню\r Работа в 1с Трактир,2012-04,2012-11
1,36c13102-4a62-477b-a475-f9a5b1c48218,Заведующий складом,Оконный завод,руководство складом и цехом по производству комплектующих,2014-02,2015-09
2,36c13102-4a62-477b-a475-f9a5b1c48218,старший кладовщик,"ООО "" Керамика сервис""",Полное ведение складского учета отдельно стоящего комплекса.,2012-03,2013-06
3,36c13102-4a62-477b-a475-f9a5b1c48218,бухгалтер складского учета,"Многопрофильный холдинг ""Вероника"" СПБ","Cклад, бухгалтер по учету ТМЦ и ОС, проведение инвентаризаций и обработка результатов, оприходование и списание.",2010-12,2011-05
4,85bfe532-1108-4142-bae5-29818bfc70a0,Старший официант,"ДВЕ ПАЛОЧКИ, сеть ресторанов свободного полета",•\tПодготовка ресторана к открытию;\r •\tВстреча и обслуживание гостей;\r •\tКонсультирование гостей об особенностях блюд и напитков; \r •\tСервировка столов;\r •\tКонтроль качества;\r •\tКонтроль чистоты в зале;\r •\tРешение конфликтных ситуаций;\r •\tСоставление графиков и расписания для официантов;\r •\tКонтроль выдачи заказа и скорости обслуживания;\r •\tКонтроль качества обслуживания гостей;\r •\tФормирование зон работы;\r •\tКонтроль работы персонала на смене\r •\tПроведение тренингов/собраний.,2008-12,2011-02
...,...,...,...,...,...,...
99995,df2f6158-c03b-4545-a4c5-e5bf80fd5a5b,Администратор ресторана,"Евразия, сеть ресторанов и суши-баров",Контроль работы ресторана,2014-06,2014-12
99996,df2f6158-c03b-4545-a4c5-e5bf80fd5a5b,Официант,jazz bar 48 chairs,Обслуживание гостей по всем европейским стандартам сервиса.,2011-06,2014-06
99997,df2f6158-c03b-4545-a4c5-e5bf80fd5a5b,"Суши-повар, официант, администратор",ООО Страйк,На поседней должности-контроль работы ресторана.,2007-04,2011-05
99998,7b0b9de2-d04c-42d5-a59d-6808b5abef8a,Член бригады ресторана,Макдоналдс,"Кассир, повар, грузчик.",2013-10,2014-11


In [5]:
display(vac_frame)

,id,name,area.name,city,company.id,company,company_link,publication_date,salary_from,salary_currency,employment.name,schedule.name,experience.name,key_skills,specializations,description
0,17016760,Официант,Санкт-Петербург,Санкт-Петербург,1905558,2Т,https://hh.ru/employer/1905558,2016-06-16T11:22:58+0300,30000.0,RUR,Полная занятость,Полный день,Нет опыта,NaN,22.193 15.313,"Мы ищем активного, жизнерадостного и дружелюбного официанта. Нашей небольшой, но очень дружной и сплоченной команде не хватает 1 человека. Требования: - опыт работы желателен, но это совсем не главное, мы обращаем внимание на личные качества, такие как вежливость и ответственность, смотрим насколько быстро вы обучаетесь и запоминаете новое, есть ли у вас интерес к профессии, желание стараться и все время стремиться к большему. Условия: - мы готовы научить вас высокому сервису, рассказать об основах гастрономии, барной культуре, психологии общения с гостем и работе в коллективе. - график работы с 07-19/ 11-23 (при составлении графика, готовы учитывать ваши пожелания) - заработная плата 2 раза в месяц - различные бонусы и премии Если у вас мало или совсем нет опыта, мы предложим вам программу стажировки, при которой вы сможете работать неполные смены."
1,17759717,Бармен-официант,Санкт-Петербург,unknown,2440865,3Ф Групп,https://hh.ru/employer/2440865,2016-07-15T13:00:14+0300,30000.0,RUR,Полная занятость,Полный день,Нет опыта,Ответственность | Материальная ответственность | Навыки межличностного общения | Работа в команде | Грамотная речь,22.175 22.193,"Обязанности: Приготовление напитков в баре Прием и подача заказов по столикам Работа с кассой Требования: Желание работать и зарабатывать. Опыт работы на подобной должности будет Вашим преимуществом. А вообще всему научим и всему обучим, так что не нужно бояться приходить без опыта работы). Наличие медицинской книжки будет Вашим преимуществом. Условия: График сменный 2\2. Будние дни с 9-00 до 21-00. Выходные дни с 9-00 до 21-00. Оформление по ТК. Оплата: 100 руб/час + % + чаевые"
2,17816582,Администратор ресторана,Санкт-Петербург,Санкт-Петербург,1911500,4ЕС,https://hh.ru/employer/1911500,2016-07-20T17:20:48+0300,45000.0,RUR,Полная занятость,Сменный график,От 3 до 6 лет,NaN,22.329,"Ресторан ""Рандеву"" приглашает на работу администратора зала с опытом работы на этой должности не менее 3-х лет. Обязанности: Контроль работы персонала ресторана Контроль качества предоставляемых услуг и соблюдения сервиса при обслуживании гостей Проведение банкетных мероприятий Обучение и тренинги персонала Создание дружелюбной атмосферы Требования: Обязательны знание стандартов сервиса, организаторские способности, ответственность, коммуникабельность, хорошо поставленная речь. Условия: График работы: вс-чт с 13-до 01-00, пт-сб с 13 до 03-00 Служебное питание."
3,17746005,Кладовщик,Санкт-Петербург,посёлок Шушары,141100,4Ф,https://hh.ru/employer/141100,2016-08-13T09:26:06+0300,38000.0,RUR,Полная занятость,Сменный график,От 1 года до 3 лет,NaN,21.275 21.563,"Обязанности: Прием товара (Кока-Кола) на склад от поставщиков; Распределение принятой продукции по адресам стеллажного хранения до 6 яруса (11,5 метров); Инвентаризация, внесение корректировок в базе данных; Прием возвратной продукции от водителей. Требования: Опыт работы кладовщиком от года; Опыт работы на электроштабеллере (желательно); Опытный пользователь ПК; Ответственность, внимательность. Условия: Оформление по ТК РФ; Своевременные выплаты заработной платы; График работы 1/1 с 20.00 до 20.00 (выходные суббота и воскресенье); Место работы: Московское шоссе 177 А (служебная развозка от м. Купчино, м. Пр.Ветеранов)."
4,17431972,Кладовщик,Санкт-Петербург,поселок Шушары,141100,4Ф,https://hh.ru/employer/141100,2016-07-12T15:22:23+0300,30000.0,RUR,Полная занятость,Полный день,От 1 года до 3 лет,NaN,21.563 21.564,Обязанности: отпуск и прием на складе ТМЦ ведение учета в программе 1С сборка наборов для магазинов помощь в погрузо-разгрузочных работах Требования: опыт работы на аналогичн

## JSON

In [158]:
data_json = {
    "17735441": {
        "common": {
            "q0": "Кладовщик",
            "q1": "Санкт-Петербург",
            "q2": "Санкт-Петербург",
            "q3": "Партия еды",
            "q4": "30000",
            "q5": ["Полная занятость"],
            "q6": ["Полный день"],
            "q7": "1-3",
            "q8": 1460,
            "q9": [
                "знание складских операций и учета владение ПК на уровне уверенного пользователя (пакет офисных программ, 1С)",
                "оперативность в работе",
                "опыт работы с продуктами питания",
                "ответственность",
                "добросовестность",
                "внимательность",
                "аккуратность"
            ],
            "q10": [
                "приемка выдача товаров",
                "ответственность за качество входящих товаров",
                "внесение первичной документации в 1С",
                "организация хранения и учета",
                "отслеживание остатков на складах",
                "участие в проведении инвентаризации"
            ],
            "q11": [
                "трудоустройство по ТК РФ",
                "«белая» заработная плата",
                "график работы 5/2, с 10:00 до 19:00",
                "молодой, позитивный и амбициозный коллектив",
                "возможность учиться и развиваться в своей профессиональной области"
            ]
        }
    },

    "17006309": {
        "common": {
            "q0": "Кладовщик",
            "q1": "Санкт-Петербург",
            "q2": "Санкт-Петербург",
            "q3": "Центральная Медицинская База",
            "q4": "30000",
            "q5": ["Полная занятость"],
            "q6": ["Полный день"],
            "q7": "1-3",
            "q8": 1460,
            "q9": [
                "среднее образование",
                "знание программы 1С.8 обязательно!",
                "честность",
                "ответственность",
                "исполнительность"
            ],
            "q10": [
                "прием, хранение и отпуск товарно-материальных ценностей на складе",
                "организация и контроль порядка в местах хранения товарно-материальных ценностей",
                "проверка правильности и ведение учета документов в программе 1С",
                "участие в инвентаризациях"
            ],
            "q11": [
                "оформление по ТК",
                "рабочий день с 8.00 до 16.30",
                "место работы: склад м. Нарвская / Балтийская"
            ]
        }
    },
    
    "20359550": {
        "common": {
            "q0": "Кладовщик",
            "q1": "Санкт-Петербург",
            "q2": "Санкт-Петербург",
            "q3": "Элтехника, производственное объединение",
            "q4": "35000",
            "q5": ["Полная занятость"],
            "q6": ["Полный день"],
            "q7": "1-3",
            "q8": 1095,
            "q9": [
                "Необходимые знания и навыки: работа с грузоподъемной техникой (эл. штабелер), транспортная тележка, способы комплектования по накладным, работа с вычислительной техникой",
                "Уверенный пользователь ПК",
                "Знание программного обеспечения Microsoft Office, (желательно MBS “Axapta”)",
                "Водительские права на управление электропогрузчиком"
            ],
            "q10": [
                "Организация погрузо-разгрузочных работ",
                "Приемка , учет МПЗ по количеству и первичным документам",
                "Составление Актов о браке МПЗ",
                "Маркировка МПЗ на участках хранения",
                "Учет МПЗ на Складе",
                "Проведение инвентаризаций"
            ],
            "q11": [
                "Работа на крупном производственном электротехническом предприятии",
                "Официальное оформление по ТК РФ, социальные гарантии",
                "Бесплатное питание, возможность повышения квалификации"
            ]
        }
    },
    
    "25282392": {
        "common": {
            "q0": "Кладовщик",
            "q1": "Санкт-Петербург",
            "q2": "Санкт-Петербург",
            "q3": "Гаврилко Вячеслав Владимирович",
            "q4": "30000",
            "q5": ["Полная занятость"],
            "q6": ["Полный день"],
            "q7": "1-3",
            "q8": 730,
            "q9": [
                "Ответственное отношение к работе, готовность к активной работе",
                "Желание работать и зарабатывать",
                "Пунктуальность, аккуратность, честность, обучаемость, хорошие адаптивные качества, готовность следовать установленным правилам и процедурам",
                "Умение выполнять несколько дел одновременно, высокая концентрация и уровень распределения внимания",
                "Способность быстро принимать решения и реагировать на информацию"
            ],
            "q10": [
                "Приемка товара на складе",
                "Размещение товара на места хранения",
                "Сборка заказов с мест хранения",
                "Упаковка заказов для отгрузки"
            ],
            "q11": [
                "заработная плата состоит из оклада и переменной части обучение в процессе работы"
            ]
        }
    },
    
    "26634319": {
        "common": {
            "q0": "Кладовщик",
            "q1": "Санкт-Петербург",
            "q2": "Санкт-Петербург",
            "q3": "Ленинградский, ТД",
            "q4": "45000",
            "q5": ["Полная занятость"],
            "q6": ["Полный день"],
            "q7": "1-3",
            "q8": 730,
            "q9": [
                "Опыт работы кладовщиком не менее 1 года с продуктами питания обязательно",
                "Знание и умение работать в учётной программе 1С 8",
                "Знание товарного соседства и условий хранения продовольственного сырья, тары и инвентаря, ротация товара",
                "Знание требований к документации поставщиков и складского хозяйства",
                "Пунктуальность, ответственность, умение управлять подчиненными, стрессоустойчивость"
            ],
            "q10": [
                "Прием сырья от поставщиков, организация его складирования, хранения и перемещения",
                "Ведение документооборота складского хозяйства",
                "Выдача сырья на производство",
                "Контроль за сроками и условиями хранения сырья",
                "Контроль за сертификатами качества и ветеринарными справками",
                "Складской учет, ротация товаров",
                "Проведение инвентаризации",
                "Погрузочно-разгрузочные работы в т.ч. вместе с грузчиками, постановка задач, контроль за их исполнением",
                "Контроль за остатками сырья на складах, ротация товаров по срокам поставки",
                "Контроль за выполнением требований санитарно-гигиенических норм, противопожарной безопасности и охраны труда" 
            ],
            "q11": [
                "Работа в режиме 8.00-17.00, 5-ка (выходные сб, вс)",
                "Оформление в соответствии с ТК РФ",
                "Заработная плата 2 раза в месяц",
                "Характер работы производственный, рабочее место расположено на ул. Степана Разина",
                "Полный рабочий день",
                "На территории работодателя"
            ]
        }
    },
    
    "27016670": {
        "common": {
            "q0": "Кладовщик",
            "q1": "Санкт-Петербург",
            "q2": "Санкт-Петербург",
            "q3": "Электрон, НИПК",
            "q4": "35000",
            "q5": ["Полная занятость"],
            "q6": ["Полный день"],
            "q7": "1-3",
            "q8": 730,
            "q9": [
                "Среднее специальное образование",
                "Знание складского учета",
                "Знание правил оформления прихода-расхода",
                "Опыт работы на складе с большим ассортиментом и оборотом ТМЦ",
                "Желательно наличие удостоверения на управление электроштабелером"
            ],
            "q10": [
                "Приемка ТМЦ по сопроводительным документам",
                "Комплектование ТМЦ для выдачи на производство",
                "Инвентаризация",
                "Учет ТМЦ в складской базе данных"
            ],
            "q11": [
                "Работа в уникальной российской компании полного цикла: от разработки до продажи, с уникальным авторитетным брендом как в России так и за рубежом!",
                "Квалифицированная команда профессионалов, в которой каждый может получить помощь и поддержку",
                "Обучение и развитие внутри компании, карьерный рост",
                "Официальная заработная плата, перечисление на карту",
                "Оформление в соответствии с ТК РФ"
            ]
        }
    },
    
    "25858089": {
        "common": {
            "q0": "Кладовщик",
            "q1": "Санкт-Петербург",
            "q2": "Санкт-Петербург",
            "q3": "Хорошие Потолки",
            "q4": "30000",
            "q5": ["Полная занятость"],
            "q6": ["Полный день"],
            "q7": "нет опыта",
            "q8": 730,
            "q9": [
                "Внимательность, высокая скорость работы",
                "Ответственность"
            ],
            "q10": [
                "Комплектация и выдача заказов для оптовых и розничных клиентов",
                "Инвентаризация",
                "Выполнение погрузочно-разгрузочных работ"
            ],
            "q11": [
                "Чистый цех, удобные условия работы, дружный коллектив",
                "Официальное оформление",
                "З/п без задержек",
                "График работы: пятидневная рабочая неделя с 09.00 до 18.00 + дежурные субботы"
            ]
        }
    },
    
    "18690366": {
        "common": {
            "q0": "Кладовщик",
            "q1": "Санкт-Петербург",
            "q2": "Санкт-Петербург",
            "q3": "Группа компаний AGAVA-S",
            "q4": "30000",
            "q5": ["Полная занятость"],
            "q6": ["Полный день"],
            "q7": "1-3",
            "q8": 1460,
            "q9": [
                "Опыт работы в аналогичной должности обязателен",
                "Знание программы 1С Склад, ПК MC Office",
                "Образование не ниже среднего профессионального",
                "Ответственность, внимательность, исполнительность, доброжелательность",
                "Гражданство РФ"
            ],
            "q10": [
                "Подбор товара по накладным, размещение товара по местам хранения",
                "Разгрузо-погрузочные работы, уборка рабочих мест",
                "Контроль и обеспечение приемки, выгрузки, складирования, отгрузки материалов и комплектующих",
                "Обеспечение сохранности материальных ценностей",
                "Учет и участие в проведении инвентаризации",
                "Осмотр и учет материальных ценностей в программе 1С"
            ],
            "q11": [
                "Оформление по ТК",
                "Оплата больничного, отпуска",
                "Корпоративная форменная одежда",
                "График работы 5/2",
                "Оплата труда: оклад+ премия по итогам работы"
            ]
        }
    },
    
    "19899949": {
        "common": {
            "q0": "Кладовщик",
            "q1": "Санкт-Петербург",
            "q2": "Санкт-Петербург",
            "q3": "Десерт Фентези",
            "q4": "35000",
            "q5": ["Полная занятость"],
            "q6": ["Сменный график"],
            "q7": "1-3",
            "q8": 1095,
            "q9": [
                "Опыт работы от 2-х лет",
                "Средне-специальное образование",
                "Наличие мед.книжки обязательно!"
            ],
            "q10": [
                "Приемка сырья от поставщиков",
                "Выдача сырья на производство",
                "Отгрузка готовой продукции",
                "Принятие участия в инвентаризации",
                "Ведение журналов входящего сырья",
                "Материальная ответственность за остатки склада",
                "Соблюдение ротации и условий хранения сырья"
            ],
            "q11": [
                "Работа на кондитерском производстве в центре города",
                "Оформление по ТК РФ",
                "График работы: 3/3 по 12 часов"
            ]
        }
    },
    
    "22830531": {
        "common": {
            "q0": "Кладовщик",
            "q1": "Санкт-Петербург",
            "q2": "Санкт-Петербург",
            "q3": "Марр Руссия, Санкт-Петербург",
            "q4": "35000",
            "q5": ["Полная занятость"],
            "q6": ["Полный день"],
            "q7": "1-3",
            "q8": 1095,
            "q9": [
                "Вежливость‚ пунктуальность‚ ответственность, мобильность",
                "Опыт работы с продуктами питания на складе от 1 года!!!!",
                "Гражданин РФ"
            ],
            "q10": [
                "Сборка заказов"
            ],
            "q11": [
                "График работы - полный рабочий день, присутствуют ночные смены 5-ти дневная рабочая неделя",
                "Официальное трудоустройство: соц. пакет, 100%\"белый\" оклад",
                "Оплачиваемый отпуск - 28 календарных дней",
                "Заработанная плата от 35000 рублей + премия по результатам работы",
                "Оплачиваемое питание",
                "Работа непосредственно с продуктами питания (прием-отпуск)"
            ]
        }
    },
}

In [159]:
with open('data.json', 'w', encoding='utf-8') as f:
    json.dump(data_json, f, ensure_ascii=False, indent=4)

In [160]:
with open('data.json', 'r') as f:
    read_json = json.load(f)

In [161]:
read_json['17735441']['common']['q9']

['знание складских операций и учета владение ПК на уровне уверенного пользователя (пакет офисных программ, 1С)',
 'оперативность в работе',
 'опыт работы с продуктами питания',
 'ответственность',
 'добросовестность',
 'внимательность',
 'аккуратность']

## Storekeeper

In [269]:
# all
klad = vac_frame[vac_frame['name'] == 'Кладовщик']

In [270]:
my_klad_list = [22966381,
 17006309,
 25282392,
 27016670,
 20359550,
 26634319,
 18690366,
 22830531,
 17735441,
 25858089]
my_klad = klad[klad['id'].isin(my_klad_list)]
my_klad = my_klad.drop_duplicates(subset=['id'])
my_klad_desc = my_klad[['id','description']]

## Algo

In [445]:
description = list(vac_frame[vac_frame["id"] == 26634319]["description"])[0]

In [446]:
description

'Обязанности: Прием сырья от поставщиков, организация его складирования, хранения и перемещения; Ведение документооборота складского хозяйства, Выдача сырья на производство; Контроль за сроками и условиями хранения сырья Контроль за сертификатами качества и ветеринарными справками; Складской учет, ротация товаров; Проведение инвентаризации; Погрузочно-разгрузочные работы в т.ч. вместе с грузчиками, постановка задач, контроль за их исполнением; Контроль за остатками сырья на складах, ротация товаров по срокам поставки; Контроль за выполнением требований санитарно-гигиенических норм, противопожарной безопасности и охраны труда. Требования: \u200b\u200b\u200b\u200bОпыт работы кладовщиком не менее 1 года с продуктами питания обязательно; Знание и умение работать в учётной программе 1С 8; Знание товарного соседства и условий хранения продовольственного сырья, тары и инвентаря, ротация товара; Знание требований к документации поставщиков и складского хозяйства; Пунктуальность, ответственност

In [447]:
tags = {
    "Обязанности": ["обязан", "задачи"],
    "Требования": ["треб", "пожелан"],
    "Условия": ["условия", "предлагаем", "предложени"]
}
bad_tags = {
    "Обязанности": [],
    "Требования": ["требований"],
    "Условия": ["условиями"]
}

In [448]:
keywords = reduce(lambda x, y: x + y, list(tags.values()))
bad_keywords = reduce(lambda x, y: x + y, list(bad_tags.values()))
all_parts = []
part = []

for word in description.split():
    if any(subword in word.lower() for subword in keywords) and (not (any(bad_subword in word.lower() for bad_subword in bad_keywords))):
        all_parts.append(part)
        all_parts.append([word])
        part = []
    else:
        part.append(word)
all_parts.append(part)

all_strings = []
for word_list in all_parts:
    all_strings.append(' '.join(word_list))

In [449]:
all_strings

['',
 'Обязанности:',
 'Прием сырья от поставщиков, организация его складирования, хранения и перемещения; Ведение документооборота складского хозяйства, Выдача сырья на производство; Контроль за сроками и условиями хранения сырья Контроль за сертификатами качества и ветеринарными справками; Складской учет, ротация товаров; Проведение инвентаризации; Погрузочно-разгрузочные работы в т.ч. вместе с грузчиками, постановка задач, контроль за их исполнением; Контроль за остатками сырья на складах, ротация товаров по срокам поставки; Контроль за выполнением требований санитарно-гигиенических норм, противопожарной безопасности и охраны труда.',
 'Требования:',
 '\u200b\u200b\u200b\u200bОпыт работы кладовщиком не менее 1 года с продуктами питания обязательно; Знание и умение работать в учётной программе 1С 8; Знание товарного соседства и условий хранения продовольственного сырья, тары и инвентаря, ротация товара; Знание требований к документации поставщиков и складского хозяйства; Пунктуальнос

In [450]:
answers_dict = {}
for string in all_strings:
    for key in tags.keys():
        if any(subword in string.lower() for subword in tags[key]) and (not (any(bad_subword in string.lower() for bad_subword in bad_keywords))):
            answers_dict[key] = all_strings[all_strings.index(string) + 1]

In [451]:
answers_dict

{'Обязанности': 'Прием сырья от поставщиков, организация его складирования, хранения и перемещения; Ведение документооборота складского хозяйства, Выдача сырья на производство; Контроль за сроками и условиями хранения сырья Контроль за сертификатами качества и ветеринарными справками; Складской учет, ротация товаров; Проведение инвентаризации; Погрузочно-разгрузочные работы в т.ч. вместе с грузчиками, постановка задач, контроль за их исполнением; Контроль за остатками сырья на складах, ротация товаров по срокам поставки; Контроль за выполнением требований санитарно-гигиенических норм, противопожарной безопасности и охраны труда.',
 'Требования': '\u200b\u200b\u200b\u200bОпыт работы кладовщиком не менее 1 года с продуктами питания обязательно; Знание и умение работать в учётной программе 1С 8; Знание товарного соседства и условий хранения продовольственного сырья, тары и инвентаря, ротация товара; Знание требований к документации поставщиков и складского хозяйства; Пунктуальность, ответ